<a href="https://colab.research.google.com/github/Bhanura/lepi_vision_lk/blob/main/LepiVisionLK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect to Google Drive & Install Libraries

In [ ]:
from google.colab import drive
import os

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Install the library for MobileNetV4
!pip install timm pandas scikit-learn

Mounted at /content/drive


Unzip the Data

In [ ]:
import zipfile

zip_path = '/content/drive/MyDrive/Lepi Vision LK/butterfly_images.zip'
extract_path = '/content/dataset'

if not os.path.exists(extract_path):
    print("Unzipping data...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Unzip complete!")
else:
    print("Data already unzipped.")

Unzipping data...
Unzip complete!


The Training Script

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import timm
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# --- CONFIGURATION ---
# Check the left sidebar folder icon in Colab to confirm this path!
BASE_DIR = '/content/dataset'
DATA_CSV = os.path.join(BASE_DIR, 'butterfly_images.csv')
IMAGE_FOLDER = os.path.join(BASE_DIR, 'butterfly_images')

MODEL_NAME = 'mobilenetv4_conv_small.e2400_r224_in1k'
BATCH_SIZE = 32
EPOCHS = 15 # You can increase this since GPU is fast
LEARNING_RATE = 0.001

# Setup GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on: {device}")

# --- DATASET CLASS ---
class ButterflyDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        img_name_base = row.iloc[0]
        label = row.iloc[-1] # Ensure we take the encoded label

        image = None
        extensions = ['.jpg', '.jpeg', '.png'] # Try common image extensions
        for ext in extensions:
            img_path = os.path.join(self.root_dir, img_name_base + ext)
            try:
                image = Image.open(img_path).convert("RGB")
                break # Image found and opened, break from extension loop
            except FileNotFoundError:
                continue # Try next extension
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
                image = None # Mark as not loaded due to other error
                break # Stop trying other extensions for this image if another error occurs

        if image is None:
            # If no image was successfully loaded after trying all extensions
            print(f"Warning: Image {img_name_base} not found with extensions {extensions}. Skipping.")
            next_idx = (idx + 1) % len(self.dataframe)
            if next_idx == idx: # Avoid infinite loop if only one image or all are bad
                raise FileNotFoundError(f"No valid image found after trying to load {img_name_base}")
            return self.__getitem__(next_idx)

        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.long)

# --- PREPARE DATA ---
if os.path.exists(DATA_CSV):
    df = pd.read_csv(DATA_CSV)

    # Encoder
    le = LabelEncoder()
    # Assuming the names are in the 2nd column (index 1)
    df['encoded_label'] = le.fit_transform(df.iloc[:, 1])
    NUM_CLASSES = len(le.classes_)
    print(f"Classes found: {le.classes_}")

    # Split
    train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['encoded_label'])

    # Transforms
    train_transforms = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    val_transforms = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    train_loader = DataLoader(ButterflyDataset(train_df, IMAGE_FOLDER, train_transforms), batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(ButterflyDataset(val_df, IMAGE_FOLDER, val_transforms), batch_size=BATCH_SIZE)

    # --- MODEL SETUP ---
    model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=NUM_CLASSES)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # --- TRAINING LOOP ---
    print("Starting Training...")
    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {running_loss/len(train_loader):.4f} | Acc: {100 * correct / total:.2f}%")

    # Save to Colab workspace
    torch.save(model.state_dict(), "butterfly_model.pth")
    print("Training Complete. Model saved.")
else:
    print(f"Error: Could not find {DATA_CSV}. Check your folder structure.")

Training on: cuda
Classes found: ['b001' 'b002' 'b003' 'b004' 'b005' 'b006' 'b007']
Starting Training...
Epoch 1/15 | Loss: 4.8913 | Acc: 20.69%
Epoch 2/15 | Loss: 2.2546 | Acc: 58.62%
Epoch 3/15 | Loss: 2.2006 | Acc: 63.22%
Epoch 4/15 | Loss: 2.8022 | Acc: 58.62%
Epoch 5/15 | Loss: 1.8693 | Acc: 62.07%
Epoch 6/15 | Loss: 1.3564 | Acc: 64.37%
Epoch 7/15 | Loss: 1.2154 | Acc: 68.97%
Epoch 8/15 | Loss: 0.8703 | Acc: 71.26%
Epoch 9/15 | Loss: 1.2439 | Acc: 71.26%
Epoch 10/15 | Loss: 0.9332 | Acc: 75.86%
Epoch 11/15 | Loss: 0.5098 | Acc: 85.06%
Epoch 12/15 | Loss: 0.8136 | Acc: 83.91%
Epoch 13/15 | Loss: 0.5994 | Acc: 88.51%
Epoch 14/15 | Loss: 0.5835 | Acc: 83.91%
Epoch 15/15 | Loss: 0.4364 | Acc: 89.66%
Training Complete. Model saved.


Save the Model back to Google Drive

In [ ]:
import shutil

# Copy the trained model from Colab to your Google Drive folder
shutil.copy("butterfly_model.pth", "/content/drive/MyDrive/Lepi Vision LK/lepi_vision_lk.pth")
print("Model successfully saved to Google Drive!")

Model successfully saved to Google Drive!
